In [1]:
import pytrt as tp
import os
import cv2

tp.set_log_level(tp.LogLevel.Warning)

In [2]:
engine_file = "yolov5m_2.fp32.trtmodel"
if not os.path.exists(engine_file):
    tp.compile_onnx_to_file(5, tp.onnx_hub("yolov5m"), engine_file)

yolo   = tp.Yolo(engine_file, tp.YoloType.V5, 0, 0.25)

In [3]:
yolo

In [4]:
image  = cv2.imread("inference/zand.jpg")
bboxes = yolo.commit(image).get()
bboxes

[<Box: left=438.02, top=434.06, right=521.42, bottom=719.22, class_label=27, confidence=0.81457>,
 <Box: left=136.21, top=200.16, right=1110.42, bottom=713.44, class_label=0, confidence=0.72617>,
 <Box: left=750.17, top=40.92, right=1141.72, bottom=709.11, class_label=0, confidence=0.90385>]

In [5]:
yolo

In [6]:
yolo.valid

True

In [7]:
import torch
import torchvision.models as models

In [8]:
input = torch.full((5, 3, 224, 224), 0.5).cuda()
m = models.resnet18(True).eval().cuda()
trt_model = tp.from_torch(m, input)

In [9]:
tout = m(input)
rtout = trt_model(input)
(tout - rtout).abs().max()

[2021-09-03 19:41:12][warn][trt_infer.cpp:26]:NVInfer: The logger passed into createInferBuilder differs from one already provided for an existing builder, runtime, or refitter. TensorRT maintains only a single logger pointer at any given time, so the existing value, which can be retrieved with getLogger(), will be used instead. In order to use a new logger, first destroy all existing builder, runner or refitter objects.



tensor(8.3447e-06, device='cuda:0', grad_fn=<MaxBackward1>)

In [10]:
import time

In [11]:
ntest = 100
t0 = time.time()
with torch.no_grad():
    for i in range(ntest):
        tout = m(input)
fee = time.time() - t0
print(f"{fee*1000:.2f} ms")

277.44 ms


In [12]:
ntest = 100
t0 = time.time()
with torch.no_grad():
    for i in range(ntest):
        tout = trt_model(input)
fee = time.time() - t0
print(f"{fee*1000:.2f} ms")

197.83 ms


In [13]:
tp.convert_torch_to_trt?

Object `tp.convert_torch_to_trt` not found.
